# RnD-3: автоконфиг гомогенности

In [1]:
import random, math, statistics
random.seed(123)

In [2]:
def gen(n,k,rho):
    data=[]
    for _ in range(n):
        c=random.gauss(0,1)
        data.append([math.sqrt(rho)*c+math.sqrt(1-rho)*random.gauss(0,1) for _ in range(k)])
    return data

def max_smd(data,g):
    out=[]
    for j in range(len(data[0])):
        a=[r[j] for r,t in zip(data,g) if t==0]
        b=[r[j] for r,t in zip(data,g) if t==1]
        v=(statistics.pvariance(a)+statistics.pvariance(b))/2
        out.append(0 if v<=0 else abs((statistics.mean(a)-statistics.mean(b))/math.sqrt(v)))
    return max(out)


In [3]:
policy=[]
for n,k,rho in [(500,12,0.2),(700,20,0.3),(900,30,0.4)]:
    data=gen(n,k,rho)
    vals=[]
    for _ in range(60):
        g=[random.randint(0,1) for _ in range(n)]
        vals.append(max_smd(data,g))
    vals=sorted(vals)
    q=0.1; tau=vals[int(q*len(vals))]
    max_iter=math.ceil(math.log(1-0.95)/math.log(1-q))
    policy.append({'N':n,'K':k,'tau':round(tau,3),'max_iter':max_iter,'top_M':min(20,max(5,round(k**0.5)))})
print('policy_table:')
for p in policy: print(p)

policy_table:
{'N': 500, 'K': 12, 'tau': 0.131, 'max_iter': 29, 'top_M': 5}
{'N': 700, 'K': 20, 'tau': 0.12, 'max_iter': 29, 'top_M': 5}
{'N': 900, 'K': 30, 'tau': 0.105, 'max_iter': 29, 'top_M': 5}


In [4]:
print('\nОценка policy:')
for p in policy:
    data=gen(p['N'],p['K'],0.3)
    succ=0; iters=[]
    for _ in range(20):
        for i in range(1,p['max_iter']+1):
            g=[random.randint(0,1) for _ in range(p['N'])]
            if max_smd(data,g)<=p['tau']:
                succ+=1; iters.append(i); break
    print({'K':p['K'],'success_rate':round(succ/20,3),'median_iter':statistics.median(iters)})


Оценка policy:
{'K': 12, 'success_rate': 1.0, 'median_iter': 3.5}
{'K': 20, 'success_rate': 1.0, 'median_iter': 3.5}
{'K': 30, 'success_rate': 0.85, 'median_iter': 11}


In [5]:
print('Рецепт: pilot max|SMD| -> tau(q) -> max_iter по target_success -> top_M по sqrt(K).')

Рецепт: pilot max|SMD| -> tau(q) -> max_iter по target_success -> top_M по sqrt(K).
